In [1]:
# Install helmet-assignment helper code
# !pip install ../input/helmet-assignment-helpers/helmet-assignment-main/ > /dev/null 2>&1
from helmet_assignment.score import NFLAssignmentScorer, check_submission
from helmet_assignment.features import add_track_features

import numpy as np
import pandas as pd
import itertools
import glob
import os
import cv2
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm
from multiprocessing import Pool
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
import random
from matplotlib.pyplot import figure
import math

In [2]:
# define platform
if os.environ['PWD'] == '/':
    ENTRY = 'input' # local run
if os.environ['PWD'] == '/kaggle/working':
    ENTRY = '../input' # kaggle run

n_test_videos = len(os.listdir(f'{ENTRY}/nfl-health-and-safety-helmet-assignment/test/'))
# Run in debug mode unless during submission
if n_test_videos == 6:
    debug = True
else:
    debug = False

# Configurables 40 1488
n_debug_samples = 1
random_state = 221
CONF_THRE = 0.3

# Read in the data.
BASE_DIR = f'{ENTRY}/nfl-health-and-safety-helmet-assignment'

labels = pd.read_csv(f'{BASE_DIR}/train_labels.csv')
if debug:
    tracking = pd.read_csv(f'{BASE_DIR}/train_player_tracking.csv')
    helmets = pd.read_csv(f'{BASE_DIR}/train_baseline_helmets.csv')
else:
    tracking = pd.read_csv(f'{BASE_DIR}/test_player_tracking.csv')
    helmets = pd.read_csv(f'{BASE_DIR}/test_baseline_helmets.csv')

In [3]:
tracking = add_track_features(tracking) # add game_play, time(dt),snap, isSnap,team,snap_offset, est_frame
tracking['label'] = tracking['player'].copy()

In [4]:
def add_cols(df): # Распарсивает video_frame в game_play и video
    df['game_play'] = df['video_frame'].str.split('_').str[:2].str.join('_')
    if 'video' not in df.columns:
        df['video'] = df['video_frame'].str.split('_').str[:3].str.join('_') + '.mp4'
    return df

def create_triangle(center, sidelength): # The area of the triangle with sidelength=10 equals 40
    a_x = int(center[0] - sidelength/2)
    b_x = int(center[0] + sidelength/2)
    c_x = int(center[0])
    
    h = int(round(math.sqrt(sidelength**2 - (sidelength/2)**2)))
    a_y = int(center[1] - h/2)
    b_y = int(center[1] - h/2)
    c_y = int(center[1] + h/2)
    
    return np.array([[a_x, a_y], [b_x, b_y], [c_x, c_y]])

def create_equilateral_triangle(center, sidelength): # The area of the triangle with sidelength=10 equals 43.3
    R = sidelength*math.sqrt(3)/3
    r = sidelength*math.sqrt(3)/6
    x = np.array([-sidelength/2, 0, sidelength/2]) + center[0]
    y = np.array([-r, R, -r]) + center[1]
    a, c, b = zip(x, y)
    return np.array([a, b, c])

In [5]:
helmets = add_cols(helmets) # Распарсиваем video_frame в game_play и video
helmets['frame'] = helmets['video_frame'].str.split('_').str[-1]

# Центры шлемов
helmets['x'] = helmets['left'] + round(helmets['width']/2).copy()
helmets['y'] = helmets['top'] + round(helmets['height']/2).copy()
helmets['y_im'] = 720 - helmets['y'].copy()

# Центры лейблов
labels['x'] = labels['left'] + round(labels['width']/2).copy()
labels['y'] = labels['top'] + round(labels['height']/2).copy()
labels['y_im'] = 720 - labels['y'].copy()

In [6]:
if debug:
    helmets = add_cols(helmets)
    labels = add_cols(labels)
    # Select `n_debug_samples` worth of videos to debug with
    sample_videos = labels['video'].drop_duplicates().sample(n_debug_samples, random_state=random_state).tolist()
    #sample_videos = ["58095_004022_Sideline.mp4"]
    sample_gameplays = ['_'.join(x.split('_')[:2]) for x in sample_videos]
    tracking = tracking[tracking['game_play'].isin(sample_gameplays)]
    helmets = helmets[helmets['video'].isin(sample_videos)]
    labels = labels[labels['video'].isin(sample_videos)]
tracking.shape, helmets.shape, labels.shape

((7106, 19), (8653, 12), (7152, 18))

In [7]:
def find_nearest(array, value): # находит значение ближайшего est_frame-а к value (фрэйму)
    value = int(value)
    array = np.asarray(array).astype(int)
    idx = (np.abs(array - value)).argmin() # из номеров est_frame вычитаем frame, берем модуль. Находим индекс минимального элемента в массиве
    return array[idx] # значение ближайшего est_frame-а

def modern_posit(image_pts, world_pts, focal_length, center):
    nb_points = np.shape(image_pts)[0]

    # centered & scaled pixel coordinates
    centered_image = np.divide(np.subtract(image_pts, center), focal_length)
    ui = centered_image[:, 0]
    vi = centered_image[:, 1]

    # homogeneous world coordinates
    homogeneous_world_pts = np.append(world_pts, np.ones((nb_points, 1)), 1)

    # pseudo inverse
    object_mat = np.linalg.pinv(homogeneous_world_pts)
    #print(object_mat)

    converged = 0
    count = 0
    t_x = 0.0
    t_y = 0.0
    t_z = 0.0
    r1 = 0.0
    r2 = 0.0
    r3 = 0.0
    while converged == 0:
        # POS part of the algorithm
        # rotation vectors
        r1_t = np.matmul(object_mat, ui)
        r2_t = np.matmul(object_mat, vi)
        # 1/t_z1 is norm of r1_t
        t_z1 = 1 / np.linalg.norm(r1_t[0:3])
        # 1/tz_2 is norm of r2_t
        t_z2 = 1 / np.linalg.norm(r2_t[0:3])
        
        # geometric average
        t_z = np.sqrt(t_z1 * t_z2)
        
        r1_n = np.multiply(r1_t, t_z)
        r2_n = np.multiply(r2_t, t_z)
        r1 = r1_n[0:3]
        r2 = r2_n[0:3]
        r3 = np.cross(r1, r2)
        r3_t = np.append(r3, t_z)
        t_x = r1_n[3]
        t_y = r2_n[3]

        # Now update the z/T z or epsilon
        # then ui, vi
        epsilon_i = np.matmul(homogeneous_world_pts, np.divide(r3_t, t_z))
        old_ui = ui
        old_vi = vi
        ui = np.multiply(epsilon_i, centered_image[:, 0])
        vi = np.multiply(epsilon_i, centered_image[:, 1])

        # check for convergence
        delta_ui = ui - old_ui
        delta_vi = vi - old_vi
        delta = np.square(focal_length) * (np.square(np.linalg.norm(delta_ui)) + np.square(np.linalg.norm(delta_vi)))

        converged = 1 if count > 0 and delta < 1 else 0
        count = count + 1
        if count > 1000:
            break

    trans = np.array([t_x, t_y, t_z], np.float64)
    rot = np.array([r1, r2, r3], np.float64)
    
    return rot, trans, count

def get_3d_angles(R):
    """
    Illustration of the rotation matrix / sometimes called 'orientation' matrix
    R = [ 
           R11 , R12 , R13, 
           R21 , R22 , R23,
           R31 , R32 , R33  
        ]

    REMARKS: 
    1. this implementation is meant to make the mathematics easy to be deciphered
    from the script, not so much on 'optimized' code. 
    You can then optimize it to your own style. 

    2. I have utilized naval rigid body terminology here whereby; 
    2.1 roll -> rotation about x-axis 
    2.2 pitch -> rotation about the y-axis 
    2.3 yaw -> rotation about the z-axis (this is pointing 'upwards') 
    https://stackoverflow.com/questions/15022630/how-to-calculate-the-angle-from-rotation-matrix
    """
    from math import asin, pi, atan2, cos 

    R11 = R[0][0]
    R12 = R[0][1]
    R13 = R[0][2]

    R21 = R[1][0]
    R22 = R[1][1]
    R23 = R[1][2]

    R31 = R[2][0]
    R32 = R[2][1]
    R33 = R[2][2]

    if R31 != 1 and R31 != -1:
        pitch_1 = -1*asin(R31)
        pitch_2 = pi - pitch_1 
        roll_1 = atan2( R32 / cos(pitch_1) , R33 /cos(pitch_1) ) 
        roll_2 = atan2( R32 / cos(pitch_2) , R33 /cos(pitch_2) ) 
        yaw_1 = atan2( R21 / cos(pitch_1) , R11 / cos(pitch_1) )
        yaw_2 = atan2( R21 / cos(pitch_2) , R11 / cos(pitch_2) ) 

        # IMPORTANT NOTE here, there is more than one solution but we choose the first for this case for simplicity !
        # You can insert your own domain logic here on how to handle both solutions appropriately (see the reference publication link for more info). 
        pitch = pitch_1 
        roll = roll_1
        yaw = yaw_1 
    else: 
        yaw = 0 # anything (we default this to zero)
        if R31 == -1: 
            pitch = pi/2 
            roll = yaw + atan2(R12,R13) 
        else: 
            pitch = -pi/2 
            roll = -1*yaw + atan2(-1*R12,-1*R13) 

    # convert from radians to degrees
    roll = roll*180/pi 
    pitch = pitch*180/pi
    yaw = yaw*180/pi 

    rxyz_deg = [roll , pitch , yaw]
    return rxyz_deg

def mapping_posit(args):
    video_frame, df = args
    gameKey,playID,view,frame = video_frame.split('_')
    gameKey = int(gameKey)
    playID = int(playID)
    frame = int(frame)
    this_tracking = tracking[(tracking['gameKey']==gameKey) & (tracking['playID']==playID)]
    est_frame = find_nearest(this_tracking.est_frame.values, frame)
    this_tracking = this_tracking[this_tracking['est_frame']==est_frame]
    len_this_tracking = len(this_tracking)
    
    helmets = df.copy()
    helmets = helmets[helmets['conf']>CONF_THRE].copy()
    if len(helmets) > len_this_tracking:
        helmets = helmets.tail(len_this_tracking)
    if len(helmets) > 22:
        helmets = helmets.tail(22)
    helmets = helmets.sort_values('x')
    #helmets = helmets.sort_values('conf', ascending=False)
    
    points_2d = np.array([helmets['x'], helmets['y_im']]).transpose()
    num_points = 3
    points_3d = np.array([this_tracking['x'], this_tracking['y'], np.zeros(len(this_tracking['x']))]).transpose()
    points_3d_low = np.array([this_tracking['x'], this_tracking['y'], (-2)*np.ones(len(this_tracking['x']))]).transpose()
    points_3d_high = np.array([this_tracking['x'], this_tracking['y'], 2*np.ones(len(this_tracking['x']))]).transpose()
    
    #helmets['dist_from_center'] = (1280 - helmets['x'])**2 + (720 - helmets['y'])**2
    #helmets = helmets.sort_values('dist_from_center', ascending=False)
    
    if len(points_2d) < 3 or len(points_3d) < 3:
        helmets['label'] = this_tracking['player'].head(len(points_2d)).values.copy()
        # print("this_tracking['player']")
        # print(this_tracking['player'])
        # print()
        # print("this_tracking['player'].head(len(points_2d))")
        # print(this_tracking['player'].head(len(points_2d)))
        # print()
        # print("helmets")
        # print(helmets)
        best_matrix = np.array([[-2.70571604e+01,  3.19350071e+03,  6.24776940e+02, -4.87844199e+04],
                                [-1.12468604e+03, -2.59801645e+02,  3.51437029e+02, 7.62795341e+04],
                                [ 0.00000000e+00,  0.00000000e+00,  9.76213969e-01, 3.94838037e+01]])
        if debug:
            these_labels = labels[(labels['gameKey']==gameKey) & (labels['playID']==playID) & (labels['frame']==frame)].copy()
            frame_scorer = NFLAssignmentScorer(these_labels, impact_weight=1000)
            frame_score = frame_scorer.score(helmets)
            helmets['frame_score'] = frame_score
            helmets['tot_dist'] = 5000
            helmets['dist_weight'] = 0
            return helmets[['video_frame', 'label', 'left','width','top','height', 'frame_int', 'frame_score', 'tot_dist', 'dist_weight']], '_'.join(['_'.join(video_frame.split('_')[:-1]), str(frame+1)]), best_matrix
        return helmets[['video_frame', 'label', 'left','width','top','height']], '_'.join(['_'.join(video_frame.split('_')[:-1]), str(frame+1)]), best_matrix
    
    #mid_points_2d = int(len(points_2d)/2)
    #points_2d_subset = points_2d[[mid_points_2d-3, mid_points_2d, mid_points_2d+3]]
    
    ######################
    ##########################
    x_mean = helmets['x'].mean()
    y_mean = helmets['y_im'].mean()
    height_mean = helmets['height'].mean()
    width_mean = helmets['width'].mean()
    #print(frame, height_mean, width_mean, height_mean*width_mean)
    #triangle = create_triangle([640, 360], 400) #height_mean*width_mean) #400
    
    triangle = create_triangle([x_mean, y_mean], height_mean*width_mean)
    points_2d_x_mat = np.repeat(points_2d[:, 0].reshape(len(points_2d), 1), repeats=3, axis=1)
    points_2d_y_mat = np.repeat(points_2d[:, 1].reshape(len(points_2d), 1), repeats=3, axis=1)
    points_triangle_x = np.repeat(triangle[:, 0].reshape(1, len(triangle)), repeats=len(points_2d), axis=0)
    points_triangle_y = np.repeat(triangle[:, 1].reshape(1, len(triangle)), repeats=len(points_2d), axis=0)
    supermat_triangle = np.sqrt(np.square(points_2d_x_mat - points_triangle_x) + np.square(points_2d_y_mat - points_triangle_y))
    all_results_triangle = {}
    tot_dist_triangle = 0.0
    supermat_triangle_shape = supermat_triangle.shape
    
    for ii in range(supermat_triangle.shape[1]):
        min_el_arg_fast = np.nanargmin(supermat_triangle)
        row_fast = min_el_arg_fast//supermat_triangle.shape[1]
        col_fast = min_el_arg_fast % supermat_triangle.shape[1]
        min_element_fast = supermat_triangle[row_fast, col_fast]#.copy()
        all_results_triangle[col_fast] = row_fast
        supermat_triangle[row_fast, :] = np.nan
        supermat_triangle[:, col_fast] = np.nan
        tot_dist_triangle += min_element_fast
    points_2d_subset = points_2d[list(all_results_triangle.values())]
    #######################
    ####################
    #points_2d_subset = points_2d[:3]
    
    
    
    all_permutations = list(itertools.permutations(range(points_3d.shape[0]), num_points))
    # print(len(all_permutations))
    # print(points_2d)
    # print(points_2d_subset)
    # print(points_3d)
    # print(view)
    min_error = float('inf')
    best_matrix = None
    posit_times = []
    for perm in all_permutations:
        points_3d_subset = points_3d[list(perm)]
        rot, trans, cnt = modern_posit(points_2d_subset, points_3d_subset, 1920, [640, 360])
        rottrans = np.append(rot, trans.reshape(3,1), axis=1)
        intrin = np.array([
            [1920, 0, 640],
            [0, 1920, 360],
            [0, 0, 1],])
        full_matrix = np.matmul(intrin, rottrans)
        
        projected_points_2d_full = np.matmul(full_matrix, np.transpose(np.append(points_3d, np.ones((points_3d.shape[0], 1)), axis=1)))
        projected_points_2d_t_full = np.transpose(projected_points_2d_full)
        projected_points_2d_homogen_full = projected_points_2d_t_full / projected_points_2d_full[2].reshape(points_3d.shape[0],1)
        
        try:
            len_proj_po = len(projected_points_2d_homogen_full[:, 0])
            proj_x_mat = np.repeat(projected_points_2d_homogen_full[:, 0].reshape(len_proj_po, 1), repeats=len(points_2d[:, 0]), axis=1)
            proj_y_mat = np.repeat(projected_points_2d_homogen_full[:, 1].reshape(len_proj_po, 1), repeats=len(points_2d[:, 0]), axis=1)
            points_2d_x_mat = np.repeat(points_2d[:, 0].reshape(1, len(points_2d[:, 0])), repeats=len_proj_po, axis=0)
            points_2d_y_mat = np.repeat(points_2d[:, 1].reshape(1, len(points_2d[:, 0])), repeats=len_proj_po, axis=0)
            supermat = np.sqrt(np.square(proj_x_mat - points_2d_x_mat) + np.square(proj_y_mat - points_2d_y_mat))

            all_results = {}
            tot_dist = 0.0
            supermat_shape = supermat.shape

            for ii in range(supermat.shape[1]):
                min_el_arg_fast = np.nanargmin(supermat)
                row_fast = min_el_arg_fast//supermat.shape[1]
                col_fast = min_el_arg_fast % supermat.shape[1]
                min_element_fast = supermat[row_fast, col_fast]#.copy()
                all_results[col_fast] = row_fast
                supermat[row_fast, :] = np.nan
                supermat[:, col_fast] = np.nan
                tot_dist += min_element_fast

            norm_error = tot_dist
            angles = get_3d_angles(full_matrix)

            if view == 'Endzone':
                if (norm_error < min_error and int(angles[0])==0 and int(angles[1])==0 and angles[2] > 0 and trans[0] > 0 and trans[1] < 0 and trans[2] > 0) or \
                   (norm_error < min_error and int(angles[0])==0 and int(angles[1])==0 and angles[2] < 0 and trans[0] < 0 and trans[1] > 0 and trans[2] > 0):
                    min_error = norm_error
                    best_matrix = full_matrix
            else:
                if norm_error < min_error and int(angles[0])==0 and int(angles[1])==0:
                    min_error = norm_error
                    best_matrix = full_matrix
        except Exception as e:
            continue
                
    ###############
    if best_matrix is None:
        best_matrix = full_matrix
    
    projected_points_2d_full = np.matmul(best_matrix, np.transpose(np.append(points_3d, np.ones((points_3d.shape[0], 1)), axis=1)))
    projected_points_2d_t_full = np.transpose(projected_points_2d_full)
    projected_points_2d_homogen_full = projected_points_2d_t_full / projected_points_2d_full[2].reshape(points_3d.shape[0],1)
    
    indexes_to_all_distances ={}
    for index in range(len(points_2d[:, 0])):
        all_distances = np.sqrt(np.square(projected_points_2d_homogen_full[:, 0] - points_2d[index, 0]) + np.square(projected_points_2d_homogen_full[:, 1] - points_2d[index, 1]))
        all_labels = list(range(len(all_distances)))
        indexes_to_all_distances[index] = sorted(list(zip(all_labels, all_distances)), key=lambda tup: tup[1])
    
    all_results = {}
    index_to_distance = {}
    tot_dist = 0.0
    expected_num_of_results = len(indexes_to_all_distances)
    for _ in range(expected_num_of_results):
        min_element = None
        min_dist = float('inf')
        min_label = None
        for k, v in indexes_to_all_distances.items():
            if v[0][1] < min_dist:
                min_dist = v[0][1]
                min_element = k
                min_label = v[0][0]
        if min_element is not None:
            all_results[min_element] = min_label
            index_to_distance[min_element] = min_dist
            tot_dist += min_dist
        indexes_to_all_distances.pop(min_element, None)
        for k, v in indexes_to_all_distances.items():
            indexes_to_all_distances[k] = [t for t in v if t[0] != min_label]
    ###############

    
    this_tracking = this_tracking.reset_index(drop=True)
    #for k, v in all_results.items():
    #    print(k,v, points_2d[k], this_tracking['player'][v], index_to_distance[k], this_tracking['x'][v], this_tracking['y'][v])
    #tgt_df = helmets.copy()
    #print(all_labels)
    #print(all_results.keys())
    ordered_labels = [this_tracking['player'][all_results[i]] for i in range(len(all_results))]
    #print(ordered_labels)
    helmets['label'] = ordered_labels
    
    ######################
    ######## finetuning ##############
    #tr_indexes = [all_results[i] for i in range(len(all_results))]
    #num_fine_points = min(len(points_2d), len(points_2d))
    best_fine_results = None
    best_tot_dist = float('inf')
    best_num_fine_points = 3
    best_index_to_distance = {}
    best_projected_points = None
    for num_fine_points in range(3, len(points_2d)+1):
        #num_fine_points = min(len(points_2d), len(points_2d))
        tr_indexes = list(all_results.values())
        pt_indexes = list(all_results.keys())
        points_3d_fine = points_3d[tr_indexes[:num_fine_points]]
        points_2d_fine = points_2d[pt_indexes[:num_fine_points]]
        # print('=====================')
        # print(tr_indexes)
        # print(points_2d_fine)
        # print(points_3d_fine)
        # print('=====================')
        rot, trans, cnt = modern_posit(points_2d_fine, points_3d_fine, 1920, [640, 360])
        rottrans = np.append(rot, trans.reshape(3,1), axis=1)
        intrin = np.array([
            [1920, 0, 640],
            [0, 1920, 360],
            [0, 0, 1],])
        best_matrix = np.matmul(intrin, rottrans)
        projected_points_2d_full = np.matmul(best_matrix, np.transpose(np.append(points_3d, np.ones((points_3d.shape[0], 1)), axis=1)))
        projected_points_2d_t_full = np.transpose(projected_points_2d_full)
        projected_points_2d_homogen_full = projected_points_2d_t_full / projected_points_2d_full[2].reshape(points_3d.shape[0],1)

        projected_points_2d_full_low = np.matmul(best_matrix, np.transpose(np.append(points_3d_low, np.ones((points_3d_low.shape[0], 1)), axis=1)))
        projected_points_2d_t_full_low = np.transpose(projected_points_2d_full_low)
        projected_points_2d_homogen_full_low = projected_points_2d_t_full_low / projected_points_2d_full_low[2].reshape(points_3d_low.shape[0],1)

        projected_points_2d_full_high = np.matmul(best_matrix, np.transpose(np.append(points_3d_high, np.ones((points_3d_high.shape[0], 1)), axis=1)))
        projected_points_2d_t_full_high = np.transpose(projected_points_2d_full_high)
        projected_points_2d_homogen_full_high = projected_points_2d_t_full_high / projected_points_2d_full_high[2].reshape(points_3d_high.shape[0],1)

        indexes_to_all_distances ={}
        for index in range(len(points_2d[:, 0])):
            all_distances = np.sqrt(np.square(projected_points_2d_homogen_full[:, 0] - points_2d[index, 0]) + np.square(projected_points_2d_homogen_full[:, 1] - points_2d[index, 1]))
            all_labels = list(range(len(all_distances)))
            indexes_to_all_distances[index] = sorted(list(zip(all_labels, all_distances)), key=lambda tup: tup[1])

        all_results_inside = {}
        index_to_distance_inside = {}
        tot_dist = 0.0
        expected_num_of_results = len(indexes_to_all_distances)
        for _ in range(expected_num_of_results):
            min_element = None
            min_dist = float('inf')
            min_label = None
            for k, v in indexes_to_all_distances.items():
                if v[0][1] < min_dist:
                    min_dist = v[0][1]
                    min_element = k
                    min_label = v[0][0]
            if min_element is not None:
                all_results_inside[min_element] = min_label
                index_to_distance_inside[min_element] = min_dist
                tot_dist += min_dist
            indexes_to_all_distances.pop(min_element, None)
            for k, v in indexes_to_all_distances.items():
                indexes_to_all_distances[k] = [t for t in v if t[0] != min_label]
        ###############
        if tot_dist < best_tot_dist:
            best_tot_dist = tot_dist
            best_fine_results = all_results_inside
            best_num_fine_points = num_fine_points
            best_index_to_distance = index_to_distance_inside
            best_projected_points = projected_points_2d_homogen_full
        
    all_results = best_fine_results
    this_tracking = this_tracking.reset_index(drop=True)
    #tgt_df = helmets.copy()
    #print(all_labels)
    #print(all_results.keys())
    ordered_labels = [this_tracking['player'][all_results[i]] for i in range(len(all_results))]
    ordered_distances = [best_index_to_distance[i] for i in range(len(all_results))]
    sig = 3
    gauss_distances = [math.exp(-(best_index_to_distance[i]**2)/(2*sig**2)) for i in range(len(all_results))]
    #for k, v in all_results.items():
    #    print(k,v, points_2d[k], this_tracking['player'][v], best_index_to_distance[k], this_tracking['x'][v], this_tracking['y'][v])
    #print(ordered_labels)
    helmets['label'] = ordered_labels
    ###########################
    ########### finetuning ############
    ###########################
    
    helmets['label'] = helmets['label'].fillna(this_tracking['player'][0])
    helmets = helmets.loc[~helmets[['label']].duplicated()]
    
    if False:
        cap = cv2.VideoCapture(f"../input/nfl-health-and-safety-helmet-assignment/train/{'_'.join(video_frame.split('_')[:-1])}.mp4")
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame-1) # optional
        success, image = cap.read()
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        fig, ax = plt.subplots(figsize=(15, 10))
        num_helmets = len(helmets)
        helmets_viz = helmets.copy()
        helmets_viz = helmets_viz.reset_index()
        for i in range(3):
            cv2.circle(image, (triangle[i][0], 720 - triangle[i][1]), 10, (0, 0, 255), 1)
        for h in range(num_helmets):
            start_point = (helmets_viz['left'][h], helmets_viz['top'][h])
            end_point = (helmets_viz['left'][h] + helmets_viz['width'][h], helmets_viz['top'][h] + helmets_viz['height'][h])
            cv2.rectangle(image, start_point, end_point, (255, 0, 0), 2)
            image = cv2.putText(image, str(int(helmets_viz['x'][h])), start_point, cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2, cv2.LINE_AA)
            
        these_labels = labels[(labels['gameKey']==gameKey) & (labels['playID']==playID) & (labels['frame']==frame)].copy()
        these_labels = these_labels.reset_index(drop=True)
        for h in range(len(these_labels)):
            start_point = (these_labels['left'][h], these_labels['top'][h])
            end_point = (these_labels['left'][h] + these_labels['width'][h], these_labels['top'][h] + these_labels['height'][h])
            cv2.rectangle(image, start_point, end_point, (0, 255, 0), 1)
            #image = cv2.putText(image, str(int(these_labels['x'][h])), start_point, cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2, cv2.LINE_AA)
        plt.imshow(image)
        plt.show()

        plt.plot(this_tracking['x'], this_tracking['y'], 'b.')
        plt.axis('scaled')
        plt.show()

        fig, ax = plt.subplots()
        ax.axis('equal')
        fig.set_size_inches((30, 30))
        ax.scatter(this_tracking['x'], this_tracking['y'], marker='.', color='b')
        for i in range(len(this_tracking)):
            ax.annotate(this_tracking['player'][i], (this_tracking['x'][i], this_tracking['y'][i]), color='b')


        fig, ax = plt.subplots()
        ax.axis('equal')
        fig.set_size_inches((30, 30))
        ax.set_xlim([these_labels['x'].min()-5,these_labels['x'].max()+5])
        ax.set_ylim([these_labels['y_im'].min()-5,these_labels['y_im'].max()+5])

        ax.scatter(points_2d[:, 0], points_2d[:, 1], marker='x', color='r')
        ax.scatter(best_projected_points[:, 0], best_projected_points[:, 1], marker='.', color='b')
        ax.scatter(these_labels['x'].values, these_labels['y_im'].values, marker='.', color='g')
        #ax.scatter(projected_points_2d_homogen_full_low[:, 0], projected_points_2d_homogen_full_low[:, 1], marker='.', color='b')
        #ax.scatter(projected_points_2d_homogen_full_high[:, 0], projected_points_2d_homogen_full_high[:, 1], marker='.', color='b')
        #print(helmets['label'])
        for i in range(len(points_2d[:, 0])):
            ax.annotate(f'{ordered_labels[i]} {gauss_distances[i]:.1f}', (points_2d[:, 0][i], points_2d[:, 1][i]), color='r', ha='right')
        

        for i in range(len(these_labels)):
            ax.annotate(these_labels['label'].values[i], (these_labels['x'].values[i], these_labels['y_im'].values[i]+10), color='g', ha='right')
        
            #ax.annotate(points_2d[:, 0][i], (points_2d[:, 0][i], points_2d[:, 1][i]), color='r', ha='right')
        for i in range(len(best_projected_points[:, 0])):
            ax.annotate(this_tracking['player'][i], (best_projected_points[:, 0][i], best_projected_points[:, 1][i]), color='b')
            #ax.annotate(this_tracking['player'][i], (projected_points_2d_homogen_full_low[:, 0][i], projected_points_2d_homogen_full_low[:, 1][i]), color='b')
            #ax.annotate(this_tracking['player'][i], (projected_points_2d_homogen_full_high[:, 0][i], projected_points_2d_homogen_full_high[:, 1][i]), color='b')

    if debug:
        these_labels = labels[(labels['gameKey']==gameKey) & (labels['playID']==playID) & (labels['frame']==frame)].copy()
        #print("these_labels")
        #print(these_labels)
        frame_scorer = NFLAssignmentScorer(these_labels, impact_weight=1000)
        frame_score = frame_scorer.score(helmets)
        helmets['frame_score'] = frame_score
        helmets['tot_dist'] = best_tot_dist
        helmets['dist_weight'] = gauss_distances
        #helmets['dist_weight'] *= (len(helmets) / best_tot_dist)
        return helmets[['video_frame', 'label', 'left','width','top','height', 'frame_int', 'frame_score', 'tot_dist', 'dist_weight']], '_'.join(['_'.join(video_frame.split('_')[:-1]), str(frame+1)]), best_matrix
    #return helmets[['video_frame','left','width','top','height', 'x', 'label']], '_'.join([str(gameKey),str(playID),str(view),str(frame+1)]), best_matrix
    return helmets[['video_frame', 'label', 'left','width','top','height']], '_'.join(['_'.join(video_frame.split('_')[:-1]), str(frame+1)]), best_matrix

def mapping_posit_even(args):
    video_frame, df, best_matrix = args
    gameKey,playID,view,frame = video_frame.split('_')
    gameKey = int(gameKey)
    playID = int(playID)
    frame = int(frame)
    this_tracking = tracking[(tracking['gameKey']==gameKey) & (tracking['playID']==playID)]
    est_frame = find_nearest(this_tracking.est_frame.values, frame)
    this_tracking = this_tracking[this_tracking['est_frame']==est_frame]
    len_this_tracking = len(this_tracking)
    
    helmets = df.copy()
    helmets = helmets[helmets['conf']>CONF_THRE].copy()
    if len(helmets) > len_this_tracking:
        helmets = helmets.tail(len_this_tracking)
    if len(helmets) > 22:
        helmets = helmets.tail(22)
    helmets = helmets.sort_values('x')
    
    points_2d = np.array([helmets['x'], helmets['y_im']]).transpose()
    points_3d = np.array([this_tracking['x'], this_tracking['y'], np.zeros(len(this_tracking['x']))]).transpose()
                
    projected_points_2d_full = np.matmul(best_matrix, np.transpose(np.append(points_3d, np.ones((points_3d.shape[0], 1)), axis=1)))
    projected_points_2d_t_full = np.transpose(projected_points_2d_full)
    projected_points_2d_homogen_full = projected_points_2d_t_full / projected_points_2d_full[2].reshape(points_3d.shape[0],1)
    
    indexes_to_all_distances ={}
    for index in range(len(points_2d[:, 0])):
        all_distances = np.sqrt(np.square(projected_points_2d_homogen_full[:, 0] - points_2d[index, 0]) + np.square(projected_points_2d_homogen_full[:, 1] - points_2d[index, 1]))
        all_labels = list(range(len(all_distances)))
        indexes_to_all_distances[index] = sorted(list(zip(all_labels, all_distances)), key=lambda tup: tup[1])
    
    all_results = {}
    tot_dist = 0.0
    expected_num_of_results = len(indexes_to_all_distances)
    index_to_distance = {}
    for _ in range(expected_num_of_results):
        min_element = None
        min_dist = float('inf')
        min_label = None
        for k, v in indexes_to_all_distances.items():
            if v[0][1] < min_dist:
                min_dist = v[0][1]
                min_element = k
                min_label = v[0][0]
        if min_element is not None:
            all_results[min_element] = min_label
            tot_dist += min_dist
            index_to_distance[min_element] = min_dist
        indexes_to_all_distances.pop(min_element, None)
        for k, v in indexes_to_all_distances.items():
            indexes_to_all_distances[k] = [t for t in v if t[0] != min_label]
    
    this_tracking = this_tracking.reset_index()
    ordered_labels = [this_tracking['player'][all_results[i]] for i in range(len(all_results))]
    helmets['label'] = ordered_labels
    
    helmets['label'] = helmets['label'].fillna(this_tracking['player'][0])
    helmets = helmets.loc[~helmets[['label']].duplicated()]
    sig = 3
    gauss_distances = [math.exp(-(index_to_distance[i]**2)/(2*sig**2)) for i in range(len(all_results))]
    
    if debug:
        these_labels = labels[(labels['gameKey']==gameKey) & (labels['playID']==playID) & (labels['frame']==frame)].copy()
        frame_scorer = NFLAssignmentScorer(these_labels, impact_weight=1000)
        frame_score = frame_scorer.score(helmets)
        helmets['frame_score'] = frame_score
        helmets['tot_dist'] = tot_dist
        helmets['dist_weight'] = gauss_distances #len(helmets) / tot_dist
        return helmets[['video_frame', 'label', 'left','width','top','height', 'frame_int', 'frame_score', 'tot_dist', 'dist_weight']], '_'.join(['_'.join(video_frame.split('_')[:-1]), str(frame+1)]), best_matrix

    return helmets[['video_frame', 'label', 'left','width','top','height']], '_'.join([str(gameKey),str(playID),str(view),str(frame+1)]), best_matrix


In [8]:
# разбиваем шлемы на каждый 4 кадр
helmets['frame_int'] = pd.to_numeric(helmets.frame, errors='coerce') # ‘coerce’ - invalid parsing will be set as NaN
odd_helmets = helmets[helmets['frame_int'] % 4 == 1].copy()
even_helmets = helmets[helmets['frame_int'] % 4 != 1].copy()
print('helmets:\t', helmets.shape)
print('odd_helmets:\t', odd_helmets.shape)
print('even_helmets:\t', even_helmets.shape)

helmets:	 (8653, 13)
odd_helmets:	 (2215, 13)
even_helmets:	 (6438, 13)


In [9]:
# # for tester
odd_helmets.to_csv('tstr/mapping_posit-odd_helmets.csv')
tracking.to_csv('tstr/mapping_posit-tracking.csv')
labels.to_csv('tstr/mapping_posit-labels.csv')

In [10]:
p = Pool(processes=4)
submission_df_list = []
frame_to_matrix = {}
df_list = list(odd_helmets.groupby('video_frame'))
# df_list = sorted(df_list, key=lambda x: int(x[0].split('_')[-1]))
with tqdm(total=len(df_list)) as pbar:
    for this_df, frame, matrix in p.imap(mapping_posit, df_list):
        submission_df_list.append(this_df)
        base_name = '_'.join(frame.split('_')[:-1])
        base_frame = frame.split('_')[-1]
        frame_to_matrix[frame] = matrix
        frame_to_matrix[base_name + '_' + str(int(base_frame) + 1)] = matrix
        frame_to_matrix[base_name + '_' + str(int(base_frame) + 2)] = matrix
        pbar.update(1)
        # if len(this_df) != len(this_df['label'].unique()):
        #     print(this_df)
        #     raise Exception("AHAHA")
p.close()

submission_df_odd = pd.concat(submission_df_list)

100%|██████████| 89/89 [02:52<00:00,  1.94s/it]


In [11]:
specific_helmets = helmets[helmets['frame_int'] == 10].copy()
out = mapping_posit((specific_helmets['video_frame'].unique()[0], specific_helmets))
out

(                     video_frame label  left  width  top  height  frame_int  \
 311002  57910_001164_Sideline_10   V33   152     16  279      18         10   
 310996  57910_001164_Sideline_10   V46   287     15  294      17         10   
 310998  57910_001164_Sideline_10   V83   411     16  221      17         10   
 310989  57910_001164_Sideline_10    V9   416     16  284      17         10   
 311000  57910_001164_Sideline_10   V11   426     18  518      18         10   
 310988  57910_001164_Sideline_10   V71   440     14  356      16         10   
 310985  57910_001164_Sideline_10   V60   445     15  334      17         10   
 310992  57910_001164_Sideline_10   V68   449     15  252      18         10   
 310993  57910_001164_Sideline_10   V66   452     13  275      18         10   
 310994  57910_001164_Sideline_10   V77   463     15  298      18         10   
 310997  57910_001164_Sideline_10   V19   470     13  128      18         10   
 310986  57910_001164_Sideline_10   H95 

In [12]:
df_list = list(even_helmets.groupby('video_frame'))
df_list = [(d[0], d[1], frame_to_matrix[d[0]]) for d in df_list]
p = Pool(processes=4)
submission_df_list = []
with tqdm(total=len(df_list)) as pbar:
    for this_df,_,_ in p.imap(mapping_posit_even, df_list):
        submission_df_list.append(this_df)
        pbar.update(1)
p.close()

submission_df_even = pd.concat(submission_df_list)

100%|██████████| 267/267 [00:03<00:00, 84.53it/s]


In [13]:
submission_total = pd.concat([submission_df_odd, submission_df_even])
#submission_total['label'] = submission_total['label'].fillna('H00')
submission_total = submission_total.loc[~submission_total[['video_frame','left', 'width', 'top', 'height']].duplicated()]
submission_total = submission_total.loc[~submission_total[['video_frame','label']].duplicated()]
#submission_total = submission_total.reset_index(drop=True)
#correct_sub = check_submission(submission_total)
# if not correct_sub:
#     raise Exception("NOT CORRECT")
ss = pd.read_csv(f'{ENTRY}/nfl-health-and-safety-helmet-assignment/sample_submission.csv')
submission_total = submission_total[ss.columns]
#submission_total.to_csv('submission.csv', index=False)

In [14]:
if debug:
    scorer = NFLAssignmentScorer(labels, impact_weight=1000)
    baseline_score = scorer.score(submission_df_odd)
    print(f"validation score {baseline_score:0.4f}")
# validation score 0.1121

if debug:
    scorer = NFLAssignmentScorer(labels, impact_weight=1000)
    baseline_score = scorer.score(submission_df_even)
    print(f"validation score {baseline_score:0.4f}")
# validation score 0.6383

if debug:
    scorer = NFLAssignmentScorer(labels, impact_weight=1000)
    baseline_score = scorer.score(submission_total)
    print("check submission correctness:", check_submission(submission_total))
    print(f"validation score {baseline_score:0.4f}")
# validation score 0.7504

validation score 0.1121
validation score 0.6383
check submission correctness: True
validation score 0.7504


In [ ]:
# if debug:
#     submission_total_sorted = submission_df_odd[['frame_int', 'frame_score']]
#     submission_total_sorted = submission_total_sorted.loc[~submission_total_sorted[['frame_int','frame_score']].duplicated()]
#     submission_total_sorted = submission_total_sorted.sort_values('frame_int')
#     figure(figsize=(10, 10))
#     plt.plot(submission_total_sorted['frame_int'][40:50], submission_total_sorted['frame_score'][40:50])
#     #plt.plot(submission_total_sorted['frame_int'], submission_total_sorted['tot_dist']/submission_total_sorted['tot_dist'].max())
#     plt.show()
#     figure(figsize=(10, 10))
#     plt.show()

In [ ]:
import sys
sys.path.append(f'{ENTRY}/easydict-master/easydict-master/')
# https://github.com/mikel-brostrom/Yolov5_DeepSort_Pytorch
sys.path.append(f'{ENTRY}/yolov5-deepsort-pytorch/Yolov5_DeepSort_Pytorch-master/Yolov5_DeepSort_Pytorch-master/deep_sort_pytorch/')
from deep_sort.deep_sort import DeepSort
from utils.parser import get_config # from deep_sort_pytorch.utils.parser import get_config

In [ ]:
%%writefile deepsort.yaml

DEEPSORT:
  REID_CKPT: "../input/yolov5-deepsort-pytorch/ckpt.t7"
  MAX_DIST: 0.2
  MIN_CONFIDENCE: 0.2
  NMS_MAX_OVERLAP: 0.5
  MAX_IOU_DISTANCE: 0.9
  MAX_AGE: 15
  N_INIT: 3
  NN_BUDGET: 30

In [ ]:
"""
Helper functions from yolov5 to plot deepsort labels.
"""

def compute_color_for_id(label):
    """
    Simple function that adds fixed color depending on the id
    """
    palette = (2 ** 11 - 1, 2 ** 15 - 1, 2 ** 20 - 1)

    color = [int((p * (label ** 2 - label + 1)) % 255) for p in palette]
    return tuple(color)

def plot_one_box(x, im, color=None, label=None, line_thickness=3):
    # Plots one bounding box on image 'im' using OpenCV
    assert im.data.contiguous, 'Image not contiguous. Apply np.ascontiguousarray(im) to plot_on_box() input image.'
    tl = line_thickness or round(0.002 * (im.shape[0] + im.shape[1]) / 2) + 1  # line/font thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(im, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    if label: # если label, то задаем его параметры
        tf = max(tl - 1, 1)  # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(im, c1, c2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(im, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)
    return im

In [ ]:
def deepsort_helmets(video_data,
                     video_dir,
                     deepsort_config='deepsort.yaml',
                     plot=False,
                     plot_frames=[]):
    # func1. Передаем датафрейм из submission_df, полученный из mapping_df. Путь к трейн или тест видео и фрэймы для печати
    
    # Setup Deepsort
    cfg = get_config()
    cfg.merge_from_file(deepsort_config)    
    deepsort = DeepSort(cfg.DEEPSORT.REID_CKPT,
                        max_dist=cfg.DEEPSORT.MAX_DIST,
                        min_confidence=cfg.DEEPSORT.MIN_CONFIDENCE,
                        nms_max_overlap=cfg.DEEPSORT.NMS_MAX_OVERLAP,
                        max_iou_distance=cfg.DEEPSORT.MAX_IOU_DISTANCE,
                        max_age=cfg.DEEPSORT.MAX_AGE,
                        n_init=cfg.DEEPSORT.N_INIT,
                        nn_budget=cfg.DEEPSORT.NN_BUDGET,
                        use_cuda=True)
    
    # Run through frames.
    video_data = video_data.sort_values('frame').reset_index(drop=True) # сортируем по порядку кадров
    ds = []
    for frame, d in tqdm(video_data.groupby(['frame']), total=video_data['frame'].nunique()): # итерируемся по кадрам. Получаем датасет каждого кадра
        d['x'] = (d['left'] + round(d['width'] / 2)) # середина по Х
        d['y'] = (d['top'] + round(d['height'] / 2)) # середина по У

        xywhs = d[['x','y','width','height']].values # значения боксов (середина по X,Y, а также длина и ширина)

        cap = cv2.VideoCapture(f'{video_dir}/{myvideo}.mp4') # берем видеофайл
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame-1) # optional. Устанавливаем индекс кадра, который будет извлекаться из видеопотока (индексация начинается от 0)
        success, image = cap.read() # Считываем кадр(картинку) , метод возвращает флаг success (True , False) и image — саму картинку (массив numpy)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Меняем цветовое пространство на RGB

        confs = np.ones([len(d),]) # создаем массив confs для каждого бокса
        clss =  np.zeros([len(d),]) # создаем массив класса для каждого бокса
        outputs = deepsort.update(xywhs, confs, clss, image) # deepsort обновляем согласно значениям боксов, confs, классы и картинку. outputs это [x1, y1, x2, y2, track_id, class_id]
#         print(outputs)
        # отрисовка промежуточных кадров с боксами из plot_frames
        if (plot and frame > cfg.DEEPSORT.N_INIT) or (frame in plot_frames): # N_INIT=3
            for j, (output, conf) in enumerate(zip(outputs, confs)): # outputs из deepsort-a
                # outputs это [x1, y1, x2, y2, track_id, class_id], распарсиваем
                bboxes = output[0:4]
                id = output[4]
                cls = output[5]
                c = int(cls)  # integer class
                label = f'{id}'
                color = compute_color_for_id(id)
                im = plot_one_box(bboxes, image, label=label, color=color, line_thickness=1)
            fig, ax = plt.subplots(figsize=(30, 20))
            video_frame = d['video_frame'].values[0]
            num_helmets = len(d)
            helmets_viz = d.copy()
            helmets_viz = helmets_viz.reset_index()
            for h in range(num_helmets):
                start_point = (helmets_viz['left'][h]-8, helmets_viz['top'][h]+15)
                end_point = (helmets_viz['left'][h] + helmets_viz['width'][h], helmets_viz['top'][h] + helmets_viz['height'][h])
                #cv2.rectangle(im, start_point, end_point, (255, 0, 0), 2)
                im = cv2.putText(im, helmets_viz['label'][h], start_point, cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 0), 1, cv2.LINE_AA)
            ax.set_title(f'Deepsort labels: {video_frame}')
            plt.imshow(im)
            plt.show()
            
        preds_df = pd.DataFrame(outputs, columns=['left','top','right','bottom','deepsort_cluster','class']) # outputs из deepsort-a, outputs это [x1, y1, x2, y2, track_id, class_id]
        if len(preds_df) > 0:
#             print("d before")
#             print(d)
            #d_better = d.copy()
            # TODO Fix this messy merge # This is similar to a left-join except that we match on nearest key rather than equal keys. Both DataFrames must be sorted by the key.
#             d = pd.merge_asof(d.sort_values(['left','top']), # исходный датафрейм этого кадра из video_data, по сути submission_df
#                               preds_df[['left','top','deepsort_cluster']] \
#                               .sort_values(['left','top']), on='left', suffixes=('','_deepsort'),
#                               direction='nearest') # Whether to search for prior, subsequent, or closest matches 

            ############### BETTER MERGE ###################
            preds_df['width'] = preds_df['right'] - preds_df['left'].copy()
            preds_df['height'] = preds_df['bottom'] - preds_df['top'].copy()
            preds_df['x'] = preds_df['left'] + round(preds_df['width']/2).copy()
            preds_df['y'] = preds_df['top'] + round(preds_df['height']/2).copy()
            preds_df['y_im'] = 720 - preds_df['y'].copy()
            d['y_im'] = 720 - d['y'].copy()

            d = d.sort_values(['left','top'])
            d = d.reset_index(drop=True)

            indexes_to_all_distances ={}
            for index in range(len(d)):
                all_distances = np.sqrt(np.square(preds_df['x'] - d['x'][index]) + np.square(preds_df['y_im'] - d['y_im'][index]))
                all_clusters = list(preds_df['deepsort_cluster'].values)
                indexes_to_all_distances[index] = sorted(list(zip(all_clusters, all_distances)), key=lambda tup: tup[1])
#             for k, v in indexes_to_all_distances.items():
#                 print(k, v[0])
            
            d['deepsort_cluster'] = [v[0][0] for k, v in indexes_to_all_distances.items()]
            #d = d_better.copy()
            ############### BETTER MERGE ###################
            
            if frame in plot_frames:
                print("d after")
                print(d[['label', 'x', 'y', 'deepsort_cluster']])
                print()

            
            
        ds.append(d)
    dout = pd.concat(ds)
    return dout


def add_deepsort_label_col(out): # на входе датафрейм из deepsort_helmets
    # Find the top occuring label for each deepsort_cluster
    sortlabel_map = out.groupby('deepsort_cluster')['label'].value_counts() \
        .sort_values(ascending=False).to_frame() \
        .rename(columns={'label':'label_count'}) \
        .reset_index() \
        .groupby(['deepsort_cluster']) \
        .first()['label'].to_dict()
    # Find the # of times that label appears for the deepsort_cluster.
    sortlabelcount_map = out.groupby('deepsort_cluster')['label'].value_counts() \
        .sort_values(ascending=False).to_frame() \
        .rename(columns={'label':'label_count'}) \
        .reset_index() \
        .groupby(['deepsort_cluster']) \
        .first()['label_count'].to_dict()
    
    out['label_deepsort'] = out['deepsort_cluster'].map(sortlabel_map)
    out['label_count_deepsort'] = out['deepsort_cluster'].map(sortlabelcount_map)

    return out


def add_deepsort_label_col_weight(out): # на входе датафрейм из deepsort_helmets
    # Find the top occuring label for each deepsort_cluster
    sortlabel_map = out.groupby(['deepsort_cluster', 'label'])['dist_weight'].sum() \
        .sort_values(ascending=False).to_frame() \
        .rename(columns={'label':'label_count'}) \
        .reset_index() \
        .groupby(['deepsort_cluster']) \
        .first()['label'].to_dict()
    # Find the # of times that label appears for the deepsort_cluster.
    sortlabelcount_map = out.groupby(['deepsort_cluster', 'label'])['dist_weight'].sum() \
        .sort_values(ascending=False).to_frame() \
        .rename(columns={'label':'label_count'}) \
        .reset_index() \
        .groupby(['deepsort_cluster']) \
        .first()['dist_weight'].to_dict()
    
    out['label_deepsort'] = out['deepsort_cluster'].map(sortlabel_map)
    out['label_count_deepsort'] = out['deepsort_cluster'].map(sortlabelcount_map)

    return out

def score_vs_deepsort(myvideo, out, labels):
    # Score the base predictions compared to the deepsort postprocessed predictions.
    myvideo_mp4 = myvideo + '.mp4'
    labels_video = labels.query('video == @myvideo_mp4')
    scorer = NFLAssignmentScorer(labels_video)
    out_deduped = out.groupby(['video_frame','label']).first().reset_index()
    base_video_score = scorer.score(out_deduped)
    
    out_preds = out.drop('label', axis=1).rename(columns={'label_deepsort':'label'})
    print(out_preds.shape)
    out_preds = out_preds.groupby(['video_frame','label']).first().reset_index()
    print(out_preds.shape)
    deepsort_video_score = scorer.score(out_preds)
    print(f'{base_video_score:0.5f} before --> {deepsort_video_score:0.5f} deepsort')

In [ ]:
submission_df = submission_total.copy()

In [ ]:
submission_df['video'] = submission_df['video_frame'].str.split('_').str[:3].str.join('_')
submission_df['frame'] = submission_df['video_frame'].str.split('_').str[-1].astype('int')

if debug:
    video_dir = f'{ENTRY}/nfl-health-and-safety-helmet-assignment/train/'
else:
    video_dir = f'{ENTRY}/nfl-health-and-safety-helmet-assignment/test/'

In [ ]:
#specific_videos = list(submission_df.groupby('video'))[0]
#test_out = deepsort_helmets(specific_videos[1], video_dir, plot_frames=[i for i in range(3, specific_videos[1]['frame_int'].max(), 20)])

In [ ]:
# Loop through test videos and apply. If in debug mode show the score change.
outs = []
for myvideo, video_data in tqdm(submission_df.groupby('video'), total=submission_df['video'].nunique()): # название видео и соответствующий ему датафрейм из submission_df, полученный из mapping_df
    print(f'==== {myvideo} ====') 
    if debug:
        # Plot deepsort labels when in debug mode.
        out = deepsort_helmets(video_data, video_dir, plot_frames=[]) # func1. Передаем датафрейм из submission_df, полученный из mapping_df. Путь к трейн или тест видео и фрэймы для печати
    else:
        out = deepsort_helmets(video_data, video_dir)        
    out = add_deepsort_label_col(out)
    outs.append(out)
submission_deepsort = pd.concat(outs).copy()

In [ ]:
ss = pd.read_csv(f'{ENTRY}/nfl-health-and-safety-helmet-assignment/sample_submission.csv')
# Final Checks
submission_deepsort['label_deepsort'] = submission_deepsort['label_deepsort'] \
    .fillna(submission_deepsort['label'])
submission_deepsort = submission_deepsort.drop('label', axis=1) \
    .rename(columns={'label_deepsort':'label'})[ss.columns]
# Drop duplicate labels
submission_deepsort = submission_deepsort.loc[
    ~submission_deepsort[['video_frame','label']].duplicated()]
check_submission(submission_deepsort)

submission_deepsort.to_csv('submission.csv', index=False)

In [ ]:
if debug:
    scorer = NFLAssignmentScorer(labels, impact_weight=1000)
    baseline_score = scorer.score(submission_deepsort)
    print("check submission correctness:", check_submission(submission_deepsort))
    print(f"validation score {baseline_score:0.4f}")

In [ ]:
def inspect_submission(sub, video, frame):
    cap = cv2.VideoCapture(f"../input/nfl-health-and-safety-helmet-assignment/train/57781_000252_Sideline.mp4")
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame-1)
    success, image = cap.read()
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    fig, ax = plt.subplots(figsize=(30, 20))
    
    these_labels = labels[(labels['gameKey']==57781) & (labels['playID']==252) & (labels['frame']==frame)].copy()
    these_labels = these_labels.reset_index(drop=True)
    for h in range(len(these_labels)):
        start_point = (these_labels['left'][h], these_labels['top'][h])
        end_point = (these_labels['left'][h] + these_labels['width'][h], these_labels['top'][h] + these_labels['height'][h])
        if these_labels['isDefinitiveImpact'][h]:
            cv2.rectangle(image, start_point, end_point, (0, 255, 0), 1)
        else:
            cv2.rectangle(image, start_point, end_point, (255, 0, 0), 1)
        image = cv2.putText(image, these_labels['label'][h], start_point, cv2.FONT_HERSHEY_SIMPLEX, 0.40, (255, 0, 0), 0, cv2.LINE_AA)
    
    these_predictions = sub[sub['video_frame']=='57781_000252_Sideline_' + str(frame)].copy()
    num_helmets = len(these_predictions)
    helmets_viz = these_predictions.copy()
    helmets_viz = helmets_viz.reset_index()
    for h in range(num_helmets):
        start_point = (helmets_viz['left'][h], helmets_viz['top'][h])
        end_point = (helmets_viz['left'][h] + helmets_viz['width'][h], helmets_viz['top'][h] + helmets_viz['height'][h])
        #cv2.rectangle(image, start_point, end_point, (255, 0, 0), 2)
        image = cv2.putText(image, helmets_viz['label'][h], (start_point[0], start_point[1] + helmets_viz['height'][h]//2), cv2.FONT_HERSHEY_SIMPLEX, 0.40, (0,255,0), 0, cv2.LINE_AA)
    plt.imshow(image)
    plt.show()


In [ ]:
if debug:
    inspect_submission(submission_deepsort, '57781_000252_Sideline', 200)